In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [2]:
base_path = "../../purple-hawk-data"
source_path = f"{base_path}/source"
dest_path = f"{base_path}/db-copy"

In [3]:
building = gpd.read_file(f"{source_path}/building.csv")

In [4]:
building = building.drop_duplicates(subset=["BIN"])

In [5]:
building_spatial = building.reindex(
    ["BIN", "CNSTRCT_YR", "HEIGHTROOF", "GROUNDELEV", "FEAT_CODE", "the_geom"], axis=1
)

In [6]:
building_spatial["geometry"] = building_spatial.the_geom.apply(wkt.loads)
building_spatial.drop("the_geom", axis=1, inplace=True)
building_wgs84 = gpd.GeoDataFrame(building_spatial, geometry="geometry", crs=4326)

In [7]:
building_lift = building_wgs84.to_crs(2263)

In [8]:
building_export = pd.DataFrame(data=building_lift)
building_export.drop("geometry", axis=1, inplace=True)
building_export["lift"] = building_lift["geometry"]
building_export["wgs84"] = building_wgs84["geometry"]

In [9]:
building_export.to_csv(f"{dest_path}/building_v2.csv", index=False)